In [2]:
# Import library-library
import os
import glob
import string

# Data Preparation and Preprocessing
import pandas as pd
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

# Word Embedding
import joblib
from sklearn.feature_extraction.text import TfidfVectorizer
import scipy.sparse

from sklearn.metrics.pairwise import cosine_similarity

# Input and Expansion Query
from textblob import TextBlob
from nltk.tokenize import wordpunct_tokenize
#from googletrans import Translator

#Split to train and test
from sklearn.model_selection import train_test_split

import mysql.connector
from mysql.connector import errorcode

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\RayRoyy\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
excluded_words = ["tempat", "waktu", "gempa", "banjir"]

NLTK_StopWords = stopwords.words('indonesian')
NLTK_StopWords.extend(["detik", "detikjatim", "detikjateng", "detikjabar", "detiksulsel", "detiksumbar", "detikbali", "detikpapua", "detiksulteng", "detikmaluku", "detjatim", "detikcom", "allahumma", "aamiin", "aamiin", "allah", "bismillah"])
NLTK_StopWords.extend(["yg", "dg", "rt", "dgn", "ny", "d", 'klo', 
                       'kalo', 'amp', 'biar', 'bikin', 'bilang', 
                       'gak', 'ga', 'krn', 'nya', 'nih', 'sih', 
                       'si', 'tau', 'tdk', 'tuh', 'utk', 'ya', 
                       'jd', 'jgn', 'sdh', 'aja', 'n', 't', 
                       'nyg', 'hehe', 'pen', 'u', 'nan', 'loh', 'rt',
                       '&amp', 'yah'])
txt_stopword = pd.read_csv("stopwords.txt", names= ["stopwords"], header = None)

# convert stopword string to list & append additional stopword
NLTK_StopWords.extend(txt_stopword["stopwords"][0].split(' '))
NLTK_StopWords = set(NLTK_StopWords)

In [4]:
def preprocessing(berita):
    # Preprocessing
    s = str(berita)
    s = s.lower()
    s = s.replace('\n', ' ')
    s = s.replace('\r', ' ')
    s = re.sub(r'[^a-zA-Z0-9\s]', ' ', s)
    tokens = [token for token in s.split(" ") if token != ""]

    # Stopwords checking
    T = [t for t in tokens if ((t in excluded_words) or (t not in NLTK_StopWords))]
    return T

In [8]:
# df =pd.read_csv("news.csv")
# df.info()
# df.head()
dataframe = []

try:
        cnx = mysql.connector.connect(user = 'root', password='Password', database = 'Petakabar')
        cursor = cnx.cursor()

        ambildata = ("SELECT berita_title, berita_date, berita_desc, berita_source FROM berita where berita_topik_id = 1")
        cursor.execute(ambildata)
        dataa = cursor.fetchall()
        for row in dataa:
            dataframe.append(row)
        
except mysql.connector.Error as err:
        if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
            print("Something is wrong with your user name or password")
        elif err.errno == errorcode.ER_BAD_DB_ERROR:
            print("Database does not exist")
        else:
            print(err)
else:
    cursor.close()
    cnx.close()

In [9]:
import pandas as pd
writer = pd.DataFrame(dataframe, columns=[
                    'Title','Date','Description', 'Source'], index=None)
writer.to_csv('news.csv', index=False, sep=',')

In [10]:
df =pd.read_csv("news.csv")
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1870 entries, 0 to 1869
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Title        1870 non-null   object
 1   Date         1870 non-null   object
 2   Description  1870 non-null   object
 3   Source       1870 non-null   object
dtypes: object(4)
memory usage: 58.6+ KB


,Title,Date,Description,Source
0,Perbarindo Peduli Beri Bantuan kepada Korban G...,"Sabtu, 10 Des 2022 22:39 WIB",Gempa yang melanda Kabupaten Cianjur terjadi p...,www.tribunnews.com
1,15 Jenazah Korban Gempa Cianjur Belum Terident...,"Senin, 12 Des 2022 19:36 WIB","15 jenazah korban gempa Cianjur, Jawa Barat, b...",www.tribunnews.com
2,"Gempa Bali M 5,2, Analisis BMKG Gempa Dipicu S...","Rabu, 14 Des 2022 08:02 WIB","Gempa bumi mengguncang wilayah Karangasem, Bal...",www.kompas.com
3,"Banyuwangi Bakal Punya Wisata Baru, Namanya Sa...","Jumat, 02 Des 2022 18:24 WIB",\-\n\nDalam waktu dekat Banyuwangi akan memili...,www.detik.com
4,"Cegah Abrasi, Bibir Pantai Karangsong Indramay...","Senin, 05 Des 2022 00:30 WIB",\-\n\nBeberapa pantai di Kabupaten Indramayu d...,www.detik.com


In [12]:
column = ['Title', 'Date', 'Description', 'Source']
df_total = pd.DataFrame()

df = pd.read_csv("news.csv")
df = df.dropna(subset=['Description'], axis=0)
df.sort_values("Description", inplace = True)
df.drop_duplicates(subset ="Description", keep = False, inplace = True)

df.head()
df_train_unprocessed, df_test_unprocessed = train_test_split(df,test_size=0.1)

In [13]:
# df_first_1000 = df.head(1000)
# len(df_first_1000)
df_train, df_test = train_test_split(df,test_size=0.1)
len(df_train)


1683

In [14]:
df_train.to_csv("corpus/df_train_unprerocessed.csv" , index = None, header=True)
df_test.to_csv("corpus/df_test_unprerocessed.csv" , index = None, header=True)

In [15]:
desc_text_test_unprocessed = []

for x in range(0, df_test_unprocessed.shape[0]):
  desc_text_test_unprocessed.append(df_test_unprocessed.iloc[x, -2])

joblib.dump(desc_text_test_unprocessed, "corpus/model/desc_text_test_unprocessed.pkl")

['corpus/model/desc_text_test_unprocessed.pkl']

In [16]:
# for x in range(0, df.shape[0]):
#     text = preprocessing(df.iloc[x, -2])
#     df.iloc[x, -2] = (' '.join(text)) 

desc_text_train_unpreprocessed = []
desc_text_test_unpreprocessed = []

for x in range(0, df_train.shape[0]):
  desc_text_train_unpreprocessed.append(df_train.iloc[x, -2])

for x in range(0, df_test.shape[0]):
  desc_text_test_unpreprocessed.append(df_test.iloc[x, -2])

In [17]:
# Simpan teks asli dokumen
joblib.dump(desc_text_test_unpreprocessed, "corpus/model/desc_text_test_unpreprocessed.pkl")
joblib.dump(desc_text_train_unpreprocessed, "corpus/model/desc_text_train_unpreprocessed.pkl")

# Simpan dataframe hasil text processing sebelum TF-IDF Vectorizer
df.to_csv('corpus/dataset/df_total_unpreprocessed.csv', index=False)
df_test.to_csv('corpus/dataset/df_test_unpreprocessed.csv', index=False)
df_train.to_csv('corpus/dataset/df_train_unpreprocessed.csv', index=False)


In [18]:
for x in range(0, df.shape[0]):
    text = preprocessing(df.iloc[x, -2])
    df.iloc[x, -2] = (' '.join(text)) 

In [19]:
desc_text_train = []
desc_text_test = []

for x in range(0, df_train.shape[0]):
  desc_text_train.append(df_train.iloc[x, -2])

for x in range(0, df_test.shape[0]):
  desc_text_test.append(df_test.iloc[x, -2])

In [20]:
# print(df_test.info())
print("Panjang Dokumen Test : ", len(desc_text_test))

print ('-'*90)

# print(df_train.info())
print("Panjang Dokumen Train : ", len(desc_text_train))

Panjang Dokumen Test :  187
------------------------------------------------------------------------------------------
Panjang Dokumen Train :  1683


In [21]:
# Simpan teks asli dokumen
joblib.dump(desc_text_test, "corpus/model/desc_text_test.pkl")
joblib.dump(desc_text_train, "corpus/model/desc_text_train_new.pkl") #INI

# Simpan dataframe hasil text processing sebelum TF-IDF Vectorizer
df.to_csv('corpus/dataset/df_total_new.csv', index=False) #INI
df_test.to_csv('corpus/dataset/df_test.csv', index=False)
df_train.to_csv('corpus/dataset/df_train.csv', index=False)


In [22]:
# Fit transform menggunakan TF-IDF Vectorizer
tfidf_vectorizer = TfidfVectorizer()
tfidf_test_matrix = tfidf_vectorizer.fit_transform(desc_text_test)
tfidf_train_matrix = tfidf_vectorizer.fit_transform(desc_text_train)

print('tfidf_test_matrix')
print(tfidf_test_matrix.shape)
print('tfidf_train_matrix')
print(tfidf_train_matrix.shape)

tfidf_test_matrix
(187, 7420)
tfidf_train_matrix
(1683, 22607)


In [23]:
joblib.dump(tfidf_vectorizer, "corpus/vectorizer/vectorizer_new.pkl") #INI
joblib.dump(tfidf_test_matrix, "corpus/matrix/tfidf_test.pkl")
joblib.dump(tfidf_train_matrix, "corpus/matrix/tfidf_train_new.pkl") #INI

['corpus/matrix/tfidf_train_new.pkl']

### Pencarian 

In [25]:
df_total = pd.read_csv('corpus/dataset/df_total_new.csv')
df_total = df_total[pd.notnull(df_total['Description'])]
print(df_total.info())
print ('-'*90)

document_text_train= joblib.load('corpus/model/desc_text_train_new.pkl')
document_text_test= joblib.load('corpus/model/desc_text_test.pkl')
print(len(document_text_test))
print(len(document_text_train))

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1870 entries, 0 to 1869
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Title        1870 non-null   object
 1   Date         1870 non-null   object
 2   Description  1870 non-null   object
 3   Source       1870 non-null   object
dtypes: object(4)
memory usage: 73.0+ KB
None
------------------------------------------------------------------------------------------
187
1683


In [26]:
kueri='gempa'
kueri=preprocessing(kueri)
kueri= [" ".join (kueri)]
print (kueri)

tfidf_matrix =joblib.load( "corpus/matrix/tfidf_train_new.pkl" )
tfidf_vectorizer = joblib.load( "corpus/vectorizer/vectorizer_new.pkl" ) 
query_vec= tfidf_vectorizer.transform(kueri)
results=cosine_similarity(tfidf_matrix, query_vec).reshape((-1))
#print (results)
print("\n======================\n")
print("Top 10 most similar documents in corpus:")

j = 1

for i in results.argsort()[-10:][::-1]:
    print("No ID Dokumen  : ", i)
    print("Tanggal        : ", df_total.iloc[i,1])
    print("Isi berita     : ", df_total.iloc[i,2])
    print("(Score: %.4f) " % results[i])

['gempa']


Top 10 most similar documents in corpus:
No ID Dokumen  :  6
Tanggal        :  Kamis, 01 Des 2022 18:10 WIB
Isi berita     :  dai ambassador dompet dhuafa uztadz genry nuswantoro menerima amanah south east asia group agency menyerahkan bantuan korban bencana gempa bumi cianjur kamis 1 12 2022 bantuan diserahkan li direktur south east asia group agency disaksikan budi santoso kepala kantor cabang dagang ekonomi indonesia kdei taiwan kepala bidang tenaga kdei taiwan purwanti uta draja bantuan uang tunai 100 000 new taiwan dollar ntd rp 50 juta direktur south east asia group agency li berharap bantuan meringankan beban korban gempa bumi cianjur berharap bantuan meringankan beban korban gempa bumi cianjur agama kemanusiaan li keterangan pers diterima kompas com kamis 1 12 2022 senada kepala kantor dagang ekonomi indonesia kdei taiwan budi santoso berharap bantuan berguna penyintas gempa cianjur corps dai dompet dhuada cordofa menggelar pelepasan dai ambassador diberangkatkan ta